In [5]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from urllib.parse import quote

In [ ]:
""" Extract province """
# hkd_df = pd.read_csv('danh_sach_hkd.csv')
# hkd_df = hkd_df.join(hkd_df['diachi'].str.rsplit(pat=',', n=3, expand=True).rename(columns={0: 'dia_diem', 1:'tinh_thanh_pho', 2:'quan_huyen', 3: 'phuong_xa'}))
# idx = hkd_df.index[(hkd_df['quan_huyen'].isnull() | hkd_df['phuong_xa'].isnull()) & hkd_df['tinh_thanh_pho'].notnull()]
# hkd_df.at[idx, ['dia_diem', 'tinh_thanh_pho', 'quan_huyen']] = np.nan
# hkd_df.to_csv('hkd_provine.csv')
# hkd_df[(hkd_df['quan_huyen'].isnull() | hkd_df['phuong_xa'].isnull()) & hkd_df['tinh_thanh_pho'].notnull()]

In [ ]:
""" Reformat db """
# sqlEngine = create_engine("mysql+pymysql://root:@127.0.0.1:3306/bid")
# query_gdt_all = "SELECT tinh, huyen, xa, name, mst, dia_chi as dia_chi_kd, nganh_nghe FROM gdt_tax_all"
# gdt_all_df = pd.read_sql(query_gdt_all, con=sqlEngine)

# query_gdt_changed = "SELECT tinh, huyen, xa, name, mst, dia_chi_kd_moi as dia_chi_kd, nganh_nghe_moi as nganh_nghe FROM gdt_tax_changed"
# gdt_changed_df = pd.read_sql(query_gdt_changed, con=sqlEngine)

# query_gdt_lowoff = "SELECT tinh, huyen, xa, name, mst, dia_chi as dia_chi_kd, nganh_nghe FROM gdt_tax_low_off"
# gdt_lowoff_df = pd.read_sql(query_gdt_lowoff, con=sqlEngine)

# query_gdt_off = "SELECT tinh, huyen, xa, name, mst, dia_chi_kd, nganh_nghe FROM gdt_tax_off"
# gdt_off_df = pd.read_sql(query_gdt_off, con=sqlEngine)

# query_gdt_without_gtgt = "SELECT tinh, huyen, xa, name, mst, dia_chi as dia_chi_kd, nganh_nghe FROM gdt_tax_without_gtgt"
# gdt_without_gtgt_df = pd.read_sql(query_gdt_without_gtgt, con=sqlEngine)

In [ ]:
# gdt_origin_df = pd.concat([gdt_all_df, gdt_changed_df, gdt_lowoff_df, gdt_off_df, gdt_without_gtgt_df])
# gdt_origin_df.drop_duplicates(subset=['mst'], inplace=True)
# gdt_origin_df.to_sql(name="gdt_origin", con=sqlEngine, if_exists="append", index=False)
# query = """UPDATE clients as f INNER JOIN temp_clients AS t ON t.company_id = f.company_id SET f.group_ids = t.group_ids WHERE f.company_id = t.company_id"""
# with sqlEngine.begin() as conn:
# 	conn.execute(query)

In [6]:
""" Read phone, email """
sqlEngine = create_engine("mysql+pymysql://root:@127.0.0.1:3306/talert")
# hkd_df = pd.read_excel("phone_mail_hkd.xls")
query_phone_mail = "select * from danh_sach_hkd"
hkd_df = pd.read_sql(query_phone_mail, con=sqlEngine)
hkd_clone =  hkd_df[hkd_df['dt'].notnull() | hkd_df['email'].notnull()]
hkd_clone.fillna('', inplace=True)
hkd_final = hkd_clone[hkd_clone['dt'].str.isnumeric()]
hkd_final.rename(columns={"dt": "phone"}, inplace=True)


/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
sql_hkd = pd.read_csv('hkd.csv')
df_all = hkd_final.merge(
            sql_hkd.drop_duplicates(),
            on=["mst"],
            how="left",
            indicator=True,
        )
df_all = df_all[df_all["_merge"] == "left_only"]
df_all.to_csv("def.csv")

In [48]:
hkd_final.to_csv("abc.csv")

In [ ]:
sqlEngine = create_engine("mysql+pymysql://root:%s@172.16.10.112:3306/hkd" % quote("Ptdl@123"), echo=True)
hkd_clone.to_sql(name="temp_client", con=sqlEngine, if_exists="replace", index=False)
query = """UPDATE gdt_origin as f INNER JOIN temp_client AS t ON t.mst = f.mst SET f.phone = t.phone, f.email = t.email WHERE f.mst = t.mst"""
with sqlEngine.begin() as conn:
	conn.execute(query)

In [53]:
hkd_final

,mst,ten,diachi,phone,email
2,8040801106,Phạm Đức Thạch,"39a Hàng Quạt,Hà Nội,Quận Hoàn Kiếm,Phường Hàn...",0837133040,
3,0109270043,Nguyễn Thị Liên,"Sạp B 280 chợ Hàng Da,Hà Nội,Quận Hoàn Kiếm,Ph...",0828226655,dangkythue2020@gmail.com
5,0102025927,Phạm Thị Hải Anh,"Kiốt 47 chợ Hàng Da,Hà Nội,Quận Hoàn Kiếm,Phườ...",048267157,
6,0109006063,Lưu Duy Khánh,"11C Phùng Hưng,Hà Nội,Quận Hoàn Kiếm,Phường Hà...",0359757676,
18,0109511820,Nguyễn Phương Loan,"45 Thuốc Bắc,Hà Nội,Quận Hoàn Kiếm,Phường Hàng Bồ",0904091872,hat11707@gmail.com
...,...,...,...,...,...
2132938,5300145090,Nguyễn Ngọc Chung,"Số nhà 028 tổ 40 phường Pom hán,Lào Cai,Thành ...",852954,
2132986,5300144996,Hà Thị Liên,"Tổ 3c Phường Bắc Lệnh,Lào Cai,Thành phố Lào Cai,",852060,
2133047,5300144107,Nguyễn văn Chung,"SN 016 Tổ 7C Pom hán,Lào Cai,Thành phố Lào Cai,",852463,
2133065,5300136201,Nguyễn thị Chung,"Tổ 37 P/Pom hán,Lào Cai,Thành phố Lào Cai,",852954,
